In [61]:
import openai
import os
import langchain as lc
from langchain.llms import OpenAI
# from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

key = os.getenv('OPENAI_API_KEY')
openai.api_key = key

In [62]:
def add_book_to_index(indexname, newbookname):
    createfile = False
    savefile = False

    # Check if filecreation needed
    if not os.path.exists(indexname):
        print("Path doesnt exist, everthing needs to be done.")
        createfile = True
        savefile = True
        # Check if Book already in index
    else:
        print("Index exists.")
        with open(indexname+'.txt','r') as f:
            if newbookname+'\n' not in f:
                print("File not in index.")
                savefile = True
            else:
                print("File already in index.")
    #Create files
    if createfile:
        open(indexname + '.txt', 'x')

    if savefile:
        loader = PyPDFLoader("PDFliterature/" + indexname + "/" + newbookname + '.pdf')
        pages = loader.load_and_split()

        if createfile:
            index = FAISS.from_documents(pages, OpenAIEmbeddings())
        else:
            index = FAISS.load_local(indexname, OpenAIEmbeddings())
            index.add_documents(pages)
            
        index.save_local(indexname)
        with open(indexname + '.txt', 'a') as f:
            f.write(newbookname + '\n')

Index exists.
File not in index.


In [ ]:
indexname = 'PhysicsLiterature'

In [ ]:
# Update Literature Index
for file in os.listdir("PDFliterature/" + indexname):
    if file.endswith(".pdf"):
        print("Adding " + file[:-4] + " to " + indexname + "index.")
        add_book_to_index(indexname, file[:-4])